<a href="https://colab.research.google.com/github/dhdbsrlw/Instruct-Tune-LLaMA-with-PEFT-Techniques/blob/main/Training_Sanity_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 초기 설정

In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!git clone https://github.com/Lightning-AI/lit-llama.git

Cloning into 'lit-llama'...
remote: Enumerating objects: 1925, done.
remote: Counting objects: 100% (664/664), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1925 (delta 566), reused 510 (delta 495), pack-reused 1261
Receiving objects: 100% (1925/1925), 1.64 MiB | 5.54 MiB/s, done.
Resolving deltas: 100% (1201/1201), done.


#**파이프라인**

In [2]:
%cd /content/drive/MyDrive/lit-llama

/content/drive/MyDrive/lit-llama


In [3]:
!pip install -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [4]:
!git-lfs install

Updated git hooks.
Git LFS initialized.


## STEP 1. 훈련 데이터셋 준비 (전처리)

한 번 데이터 저장되었으므로, 다시 실행시킬 필요 없다.

### 1-1. OpenLLaMA 모델 체크포인트 받아오기

In [11]:
%pwd

'/content/drive/MyDrive/lit-llama'

In [9]:
# 모델 체크포인트 받아오기 (실제 LLaMA 혹은 OpenLLaMA)
# 일단 OpenLLaMA 채택

# Make sure you have git-lfs installed (https://git-lfs.com): git lfs install
# ! git clone https://huggingface.co/openlm-research/open_llama_7b checkpoints/open-llama/7B


Cloning into 'checkpoints/open-llama/7B'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), 7.72 KiB | 28.00 KiB/s, done.
Filtering content: 100% (3/3), 4.55 GiB | 5.42 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/lit-llama/checkpoints/open-llama/7B/.git/hooks/post-checkout': Permission denied
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [ ]:
# Memory 이슈로 더 작은 모델 받아오기
!git clone https://huggingface.co/openlm-research/open_llama_3b checkpoints/open-llama/3B

In [ ]:
!python scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/3B --model_size 3B

### 1-2. 훈련 데이터셋 Prompt 전처리 및 Split

In [ ]:
%cd /content/drive/MyDrive/lit-llama

In [18]:
%pwd

'/content/drive/MyDrive/lit-llama'

In [22]:
!python scripts/prepare_dolly.py

# 전처리 완료된 데이터셋이 어느 경로에 저장되는지 살펴보기
# (분할 전) 전체 데이터셋은 - lit-llama/data/dolly 폴더 내 저장
# (분할 후) 데이터셋은 - lit-llama/data 폴더 내 .pt 형식으로 저장

train has 13,011 samples
val has 2,000 samples
Processing train split ...
100% 13011/13011 [00:14<00:00, 889.34it/s]
Processing test split ...
100% 2000/2000 [00:02<00:00, 857.19it/s]


## STEP 2. 파인튜닝

### 2-1. wandb 연동

In [5]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [6]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# wandb web server 와 연동
# !wandb init

### 3-2. 파인튜닝 수행

In [23]:
# 본격적으로 파인튜닝 시키기 전에, WANDB 와 연결
# 파인튜닝 코드 args 살펴보고, 적절히 설정 (pretrained_path 수정 필요)

!python finetune/lora.py --data_dir data/dolly --out_dir out/lora/dolly

wandb: Currently logged in as: dhdbsrlw. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/lit-llama/wandb/run-20231206_040447-ut6lzv1k
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run experiment_3
wandb: ⭐️ View project at https://wandb.ai/dhdbsrlw/lora
wandb: 🚀 View run at https://wandb.ai/dhdbsrlw/lora/runs/ut6lzv1k
/content/drive/MyDrive/lit-llama/finetune/lora.py:255: JsonargparseDeprecationWarning: 
    By default only one JsonargparseDeprecationWarning per type is shown. To see all warnings set environment
    variable JSONARGPARSE_DEPRECATION_WARNINGS=all and to disable the warnings set
    JSONARGPARSE_DEPRECATION_WARNINGS=off.

  from jsonargparse.cli import CLI
/content/drive/MyDrive/lit-llama/finetune/lora.py:255: JsonargparseDeprecationWarning: 
    Only use the public API as described in https://jsonargparse.readthedocs.io/en/stable/#api-reference.
    Imp

In [ ]:
!python finetune/adapter_v2.py --data_dir data/dolly --out_dir out/adapter_v2/dolly